In [10]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# import hshap
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from itertools import combinations 
# select device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [11]:
model_directory = 'h-shap-main/demo/BBBC041/'
filename = model_directory + 'model.pt'
model = torch.hub.load("pytorch/vision:v0.10.0", "resnet18", pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.load_state_dict(torch.load(filename, map_location=device))
model = model.to(device)
model.eval()
torch.cuda.empty_cache()
model = nn.Sequential(
    model,
    nn.Softmax(dim=1),
)

Using cache found in /Users/ishankalburge/.cache/torch/hub/pytorch_vision_v0.10.0


In [12]:
image_path = '/Users/ishankalburge/Documents/JOHNS-HOPKINS/3_1FA22/biomedical-data-design/generative-inpainting-pytorch/test_image.png'
image = Image.open(image_path)
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)]
)
image_t = transform(image).to(device)
prep_img = torch.unsqueeze(image_t, 0)

In [13]:
prob_scores = model(prep_img).detach().numpy()[0]
cl, baseline_v = np.argmax(prob_scores), prob_scores[np.argmax(prob_scores)]
(cl,baseline_v)

(1, 0.9542775)

In [14]:
directory = 'generative-inpainting-pytorch/output_shap'
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)]
)
playerList = []
classList = []
probList = []
for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f) and f[-4:] == '.png':
            ext = filename[6:-4]
            players = list(ext)
            players = [int(i) for i in players]
            playerList.append(players)
            print(filename[6:-4])
            image = Image.open(f)
            image_t = transform(image).to(device)
            prep_img = torch.unsqueeze(image_t, 0)
            prob_scores = model(prep_img).detach().numpy()[0][cl]
#             c, p = np.argmax(prob_scores), prob_scores[np.argmax(prob_scores)]
#             classList.append(c)
            p = prob_scores
            probList.append(p)
            print(p)

124
0.5461519
134
0.00012778222
123
0.02953154
3
0.60529083
1234
0.1125377
2
0.9991227
12
0.40260196
13
0.0044785477
1
0.79251206
4
0.9561983
234
0.3745919
14
0.6737065
24
0.9886014
23
0.9542656
34
0.0277682


In [15]:
playerList

[[1, 2, 4],
 [1, 3, 4],
 [1, 2, 3],
 [3],
 [1, 2, 3, 4],
 [2],
 [1, 2],
 [1, 3],
 [1],
 [4],
 [2, 3, 4],
 [1, 4],
 [2, 4],
 [2, 3],
 [3, 4]]

In [7]:
from math import comb

In [17]:
N = [1,2,3,4]
def shapley(N, j, base, v, players):
    shap = 0
    card = len(N)
    for S in range(card):
        print(S)
        w  = comb(card, S)**(-1)
        sets = list(combinations(N, S+1))
        for s in sets:
            print(s)
            
shapley(N,1,1,1,1)
t = (1,2,3)
3 in t

0
(1,)
(2,)
(3,)
(4,)
1
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)
2
(1, 2, 3)
(1, 2, 4)
(1, 3, 4)
(2, 3, 4)
3
(1, 2, 3, 4)


True

In [103]:
s = '123'
l = list(s)
l = [int(i) for i in l]
l

[1, 2, 3]